In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

from delta import *

In [2]:
# Create SparkSession
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

24/10/02 15:17:32 WARN Utils: Your hostname, ubuntulab resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/10/02 15:17:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntu/codigos/spark-delta/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9fca5cc3-0303-4a08-87aa-7ffedfc96120;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.4.0/delta-core_2.12-2.4.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;2.4.0!delta-core_2.12.jar (1318ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/2.4.0/delta-storage-2.4.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;2.4.0!delta-storage.jar (334ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (311ms)
:: resolution report :: resolve 4409ms :: arti

In [3]:
spark

In [4]:
spark.sql(
  """
  CREATE TABLE carro_delta (id INT, placa STRING) USING delta
  """
)

24/10/02 15:18:10 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[]

In [5]:
spark.sql("select * from carro_delta").show()

+---+-----+
| id|placa|
+---+-----+
+---+-----+



In [6]:
from delta.tables import DeltaTable

carro = DeltaTable.forPath(spark, "./spark-warehouse/carro_delta")

In [11]:
carro.history(2).show()

+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+--------------------+
|version|           timestamp|userId|userName|   operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+--------------------+
|      0|2024-10-02 15:18:...|  null|    null|CREATE TABLE|{isManaged -> tru...|null|    null|     null|       null|  Serializable|         true|              {}|        null|Apache-Spark/3.4....|
+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+--------------------+



In [12]:
spark.sql("INSERT INTO carro_delta VALUES (1, 'XYZ1J34'), (2, 'RLC5B93'), (3, 'ABV1V23')")

DataFrame[]

In [14]:
spark.sql("select * from carro_delta").show()

+---+-------+
| id|  placa|
+---+-------+
|  2|RLC5B93|
|  1|XYZ1J34|
|  3|ABV1V23|
+---+-------+



In [41]:
carro.history().show(truncate=False)

+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                                                                                                                                                                         |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppen

In [18]:
spark.sql(
    """
    alter table carro_delta add column marca STRING, modelo STRING, ano INT
    """
)

DataFrame[]

In [19]:
spark.sql(
    """
    select * from carro_delta
    """
).show()

+---+-------+-----+------+----+
| id|  placa|marca|modelo| ano|
+---+-------+-----+------+----+
|  2|RLC5B93| null|  null|null|
|  1|XYZ1J34| null|  null|null|
|  3|ABV1V23| null|  null|null|
+---+-------+-----+------+----+



In [20]:
spark.sql(
    """
    update carro_delta set marca = 'Renault', modelo = 'Sandero', ano = 2021 where id = 1
    """
)

DataFrame[num_affected_rows: bigint]

In [21]:
spark.sql(
    """
    select * from carro_delta
    """
).show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  1|XYZ1J34|Renault|Sandero|2021|
|  2|RLC5B93|   null|   null|null|
|  3|ABV1V23|   null|   null|null|
+---+-------+-------+-------+----+



In [22]:
DeltaTable.isDeltaTable(spark, "spark-warehouse/carro_delta")

True

In [26]:
spark.sql('select * from carro_delta').show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  1|XYZ1J34|Renault|Sandero|2021|
|  2|RLC5B93|     GM|tracker|2020|
|  3|ABV1V23|   null|   null|null|
+---+-------+-------+-------+----+



In [25]:
spark.sql('update carro_delta set marca="GM", modelo="tracker", ano=2020 where id = 2    ')

DataFrame[num_affected_rows: bigint]

In [27]:
spark.sql('update carro_delta set marca="Ford", modelo="EcoSport", ano=2022 where id = 3    ')

DataFrame[num_affected_rows: bigint]

In [28]:
spark.sql('select * from carro_delta').show()

+---+-------+-------+--------+----+
| id|  placa|  marca|  modelo| ano|
+---+-------+-------+--------+----+
|  1|XYZ1J34|Renault| Sandero|2021|
|  3|ABV1V23|   Ford|EcoSport|2022|
|  2|RLC5B93|     GM| tracker|2020|
+---+-------+-------+--------+----+



In [39]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [40]:
spark.sql('describe HISTORY carro_delta').show(truncate=False);

+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                                                                                                                                                                         |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppen